In [204]:
from bs4 import BeautifulSoup
import requests

In [205]:

ads_all = []
base_url = "https://www.bidbud.co.nz/search?category=0002-0356-0032-2273-&page="

page_num = 1

response = requests.get("https://www.bidbud.co.nz/search?category=0002-0356-0032-2273-&page=" + str(page_num))
html = response.content
soup = BeautifulSoup(html, "html.parser")
page_ads = soup.find_all('a', class_="listing_title")

ads_all.extend(page_ads)

while len(page_ads) > 0:
    page_num += 1
    response = requests.get("https://www.bidbud.co.nz/search?category=0002-0356-0032-2273-&page=" + str(page_num))
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    page_ads = soup.find_all('a', class_="listing_title")

    ads_all.extend(page_ads)

ads_hrefs = [a.get('href') for a in ads_all]

In [206]:
# Post clean schema perhaps?

dict_keys = ['ListingId', \
             'Title', \
             'Category', \
             'StartPrice', \
             'BuyNowPrice', \
             'StartDate', \
             'EndDate', \
             'IsFeatured', \
             'BiddersAndWatchers', \
             'MaxMidAmount', \
             'IsNew', \
             'Region', \
             'Suburb', \
             'BidCount', \
             'ViewCount', \
             'IsReserveMet', \
             'HasReserve', \
             'HasBuyNow', \
             'ReserveState', \
             'RelistedItemId', \
             'Subtitle', \
             'IsBuyNowOnly', \
             'TotalReviewCount', \
             'UnansweredQuestionCount', \
            ]

listing_dicts = [dict.fromkeys(dict_keys) for i in range(len(ads_all) + 1)]


In [208]:
listing_base_url = "https://www.bidbud.co.nz/"

listing_dicts = []

for ad in ads_hrefs:
    print(ad)
    response = requests.get("https://www.bidbud.co.nz/" + ad)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    title = soup.find('h2')

    item_info = soup.find('div', id="item_info") # Listing start/end, prices, watches

    description = soup.find('div', id='description') # Description element

    try:
        listing_attributes = description.find('table', id='listingattributes') # table with memory, HDD, screen, cores. Child of description
        attr_rows = listing_attributes.findAll('tr')
        attr_headers = []
        attr_values = []
        for row in attr_rows:
            header = row.find('th').text.strip()
            value = row.find('td').text.strip().strip(':')
            attr_headers.append(header)
            attr_values.append(value)

        description_paragraphs = description.findAll('p')
        description_paragraphs = [p.text for p in description_paragraphs]
        seller_information = description.find('div', class_='panel-body')
        try:
            member_latest_listings_href = seller_information.find('a', title="View member's listings").get('href')
            member_username = seller_information.find('a', title="View member's listings").text
        except:
            member_latest_listings_href = None
            member_username = seller_information.find('b', class_="warning").text
            print("Warning user")


        member_feedback_href = seller_information.find('a', title="View member's feedback").get('href')
        member_feedback_score = seller_information.find('a', title="View member's feedback").text

        seller_information_string = seller_information.text.strip() # Will need to clean this 
        view_count = soup.find('span', id="view_count").text
        listing_dict = dict()

        listing_dict['ListingId'] = ads_hrefs[0].strip('/')
        listing_dict['Title'] = title.text

        attrs_dict = dict(zip(attr_headers, attr_values))
        listing_dict.update(attrs_dict)

        listing_dict['Description'] = description_paragraphs

        listing_dict['MemberFeedbackUrl'] = member_feedback_href
        listing_dict['MemberFeedbackScore'] = member_feedback_score
        listing_dict['SellerInformationString'] = seller_information_string

        listing_dict['ViewCount'] = view_count

        listing_dict['ItemInfo'] = item_info.text.strip()
        listing_dicts.append(listing_dict)
    except:
        listing_dicts.append(dict())




/4760235612
/4759747694
/4756489268
/4756449848
/4754794981
/4753161032
/4751534646
/4763617222
/4763244122
/4763180046
Warning user
/4763126803
/4763052730
/4762268188
/4761730047
/4761672076
/4761208625
/4761011832
Warning user
/4760964609
/4760963360
/4760962790
/4760108533
/4759902885
/4759675516
/4759456785
/4758575072
/4758412383
/4758314516
/4757741269
/4757739762
/4757738643
/4757586903
/4756649212
/4756434598
/4756350128
/4756322678
/4756056786
/4755999977
/4755844413
/4755811583
/4755811539
/4755706628
/4754833610
/4754752110
/4754617448
/4754210907
/4753984876
/4753794306
/4753311313
/4752952033
/4752730797
/4752729915
/4752728530
/4752727732
/4752727098
/4752641648
/4752184036
/4751590745
/4751585264
/4743474132
/4755690658
/4763369020
/4763310311
/4761922537
/4759470927
/4758708326
/4758708179
/4757887164
/4757650487
/4757264480
/4757264415
/4757263955
/4756131532
/4755859582
/4754532043
/4754150822
/4752734268
/4752088679
/4750238081
/4763599495
/4763464552
/4763334465
/4

In [209]:
listing_dicts

[{'ListingId': '4760235612',
  'Title': 'Macbook Pro 15" 2018 16GB Touchbar 6core i7 512GB',
  'Memory:': '16 to 31 GB',
  'Hard Drive Size:': '500 to 999 GB',
  'Screen Size:': '15"',
  'Cores:': '6',
  'Description': ['Apple Macbook Pro 15-inch intel  2018 A1990',
   'Macbook is like in brand new condition.',
   'macOS has been installed. Ready for use. Comes with Genuine Apple charger and cable.',
   'Battery cycle count is only 1',
   'Intel i7 CPU\n16GB RAM\n512GB SSD\nTouchbar\nTouchID',
   '6 Month Warranty:\nReturn to base warranty covers hardware issues only and does not cover consumables such as battery, physical damage, water damage or misuse.',
   'We are at ferry road Waltham Christchurch \nMonday to Friday - 9:00AM to 6:00pm\nSaturday - 10:00AM to 3:00PM'],
  'MemberFeedbackUrl': 'https://www.bidbud.co.nz/members/feedback/6820672',
  'MemberFeedbackScore': '35',
  'SellerInformationString': 'techcareplus\xa0(35\xa0\xa0)Member for 5 years97.3% positive feedbackLocated in C

In [211]:
import pandas as pd

df = pd.DataFrame.from_dict(listing_dicts)

In [214]:
df.to_csv("apple_laptops_bidbud.csv")